In [1]:
import os
import flow
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import pandas as pd

In [4]:
import cascade as cas

In [6]:
cas.metadata.metadata.dirupdate_training('OA67')

In [2]:
mouse = 'OA27'
trace_type = 'dff'
start_time = -1
end_time = 6
downsample = True
verbose = True

runs = flow.metadata.RunSorter.frommeta(mice=[mouse])

# triggering parameters
start_time = start_time
end_time = end_time
trace_type = trace_type

trial_list = []
for run in runs:

    # get your t2p object
    t2p = run.trace2p()

    # trigger all trials around stimulus onsets to get trial number
    run_traces = t2p.cstraces('', start_s=start_time, end_s=end_time, trace_type=trace_type,
                    cutoff_before_lick_ms=-1, errortrials=-1, baseline=(0, -1),
                    baseline_to_stimulus=True)
    trial_idx = range(np.shape(run_traces)[2])

    # get your learning_state
    run_tags = [str(s) for s in run.tags]
    run_tags = run.tags
    if 'naive' in run_tags:
        learning_state = 'naive'
    elif 'learning' in run_tags:
        learning_state = 'learning'
    elif 'reversal1' in run_tags:
        learning_state = 'reversal1'
    elif 'reversal2' in run_tags:
        learning_state = 'reversal2'  
    learning_state = [learning_state]*len(trial_idx)

    # get hunger state for all trials, consider hungry if not sated
    if 'sated' in run_tags:
        hunger = 'sated'
    else:
        hunger = 'hungry'
    hunger = [hunger]*len(trial_idx)

    # get relevant trial-distinguising tags excluding kelly, hunger-state, and learning-state
    tags = [str(run_tags[s]) for s in range(len(run_tags)) if run_tags[s] != hunger[0]
            and run_tags[s] != learning_state[0]
            and run_tags[s] != 'kelly'
            and run_tags[s] != 'learning_start'
            and run_tags[s] != 'reversal1_start'
            and run_tags[s] != 'reversal2_start']
    if tags == []:  # define as "standard" if the run is not another option
        tags = ['standard']
    tags = [tags[0]]*len(trial_idx)

    # get trialerror ensureing you don't include runthrough at end of trials
    trialerror = np.array(t2p.d['trialerror'][trial_idx])
    
#     oris = []
#     css = []
# #     print(t2p.d['codes'].items())
#     print(t2p.d['orientations'].items())
#     for trial in t2p.d['condition'][trial_idx]:
#         # get cs and ori
#         print(trial)
#         print(t2p.d['codes'].items())
#         print(trial in t2p.d['codes'].items())
#         for k, v in t2p.d['codes'].items():
#             if v == trial:
#                 print(v)
#                 print(k)
#                 print(t2p.d['orientations'][k])
#                 css.append(k)
#                 oris.append(t2p.d['orientations'][k])
                
#     # get cs and orientation infor for each trial
    oris = []
    css = []
    for trial in t2p.d['condition'][trial_idx]:
        # get cs and ori
        try: # python 3
            codename = list(t2p.d['codes'].keys())[list(t2p.d['codes'].values()).index(trial)])
        except: # python 2
            codename = t2p.d['codes'].keys()[t2p.d['codes'].values().index(trial)]
        oriname = t2p.d['orientations'][codename]
        css.append(codename)
        oris.append(oriname)
        
        print(trial, codename, oriname)
#     # get mean running speed for time stim is on screen
#     all_onsets = t2p.csonsets()
#     all_offsets = t2p.d['offsets'][0:len(all_onsets)]
#     if t2p.d['running'].size > 0:
#         speed_vec = t2p.speed()
#         speed_vec = speed_vec.astype('float')
#         speed = []
#         for s in trial_idx:
#             try:
#                 speed.append(np.nanmean(speed_vec[all_onsets[s]:all_offsets[s]]))
#             except:
#                 speed.append(np.nan)
#         speed = np.array(speed)
#     else:
#         speed = np.full(len(trial_idx), np.nan)

#     # get offset relative to triggered data
# #     offsets = all_offsets - all_onsets + (np.abs(start_time)*np.round(t2p.d['framerate']))
# #     offsets = offsets.flatten()

#     # get ensure/ensure/firstlick relative to triggered data
#     ensure = t2p.ensure()
#     ensure = ensure.astype('float')
#     ensure[ensure == 0] = np.nan
#     ensure = ensure - all_onsets + (np.abs(start_time)*np.round(t2p.d['framerate']))

#     quinine = t2p.quinine()
#     quinine = quinine.astype('float')
#     quinine[quinine == 0] = np.nan
#     quinine = quinine - all_onsets + (np.abs(start_time)*np.round(t2p.d['framerate']))

#     firstlick = t2p.firstlick('')[trial_idx]
#     firstlick = firstlick + (np.abs(start_time)*np.round(t2p.d['framerate']))

#     # downsample all timestamps to 15Hz if framerate is 31Hz
#     if (t2p.d['framerate'] > 30) and downsample:
#         ensure = ensure/2
#         quinine = quinine/2
#         firstlick = firstlick/2

#     # create your index out of relevant variables
#     index = pd.MultiIndex.from_arrays([
#                 [run.mouse]*len(trial_idx),
#                 [run.date]*len(trial_idx),
#                 [run.run]*len(trial_idx),
#                 trial_idx
#                 ],
#                 names=['mouse', 'date', 'run', 'trial_idx'])

#     data = {'orientation':  oris, 'condition': css,
#             'trialerror': trialerror, 'hunger': hunger,
#             'learning_state': learning_state, 'tag': tags,
#             'firstlick': firstlick, 'ensure': ensure,
#             'quinine': quinine, 'speed': speed}

#     # append all trials across all runs together into a list
#     trial_list.append(pd.DataFrame(data, index=index))

#     # clear your t2p to save RAM
#     run._t2p = None
#     if verbose:
#         print('Run: ' + str(run) + ': ' + str(len(trial_list)))

# # concatenate all runs together in final dataframe
# trial_df = pd.concat(trial_list, axis=0)

#     # create folder structure if needed
# #     save_dir = os.path.join(flow.paths.outd, str(mouse))
# #     if not os.path.isdir(save_dir):
# #         os.mkdir(save_dir)

# #     # save (trace_type should be irrelevant for metadata)
# #     save_path = os.path.join(save_dir, str(runs[0].mouse) + '_df_trialmeta.pkl')
# #     trial_df.to_pickle(save_path)

24
[('orientation_0', 17), ('orientation_225', 22), ('orientation_180', 21), ('orientation_90', 19), ('orientation_270', 23), ('orientation_135', 20), ('orientation_45', 18), ('orientation_315', 24)]
False
19
[('orientation_0', 17), ('orientation_225', 22), ('orientation_180', 21), ('orientation_90', 19), ('orientation_270', 23), ('orientation_135', 20), ('orientation_45', 18), ('orientation_315', 24)]
False
23
[('orientation_0', 17), ('orientation_225', 22), ('orientation_180', 21), ('orientation_90', 19), ('orientation_270', 23), ('orientation_135', 20), ('orientation_45', 18), ('orientation_315', 24)]
False
19
[('orientation_0', 17), ('orientation_225', 22), ('orientation_180', 21), ('orientation_90', 19), ('orientation_270', 23), ('orientation_135', 20), ('orientation_45', 18), ('orientation_315', 24)]
False
22
[('orientation_0', 17), ('orientation_225', 22), ('orientation_180', 21), ('orientation_90', 19), ('orientation_270', 23), ('orientation_135', 20), ('orientation_45', 18), (

/Users/kelly_mcguire/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:69: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


3
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
5
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
4
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
3
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
5
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
6
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
4
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
4
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
4
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
4
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
4
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
3
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
5
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
5
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
6
[('neutral', 5), ('plus', 3), ('minus', 4), ('blank', 6)]
False
6
[('neutr

KeyboardInterrupt: 

In [3]:
count = 0
for run in runs:
    

#     # check for undesirable tags (runs where animal is not training)
#     tag_flag = [False for s in range(len(run.tags)) if run.tags[s] == 'sated'
#                 or run.tags[s] == 'naive'
#                 or run.tags[s] == 'orientation_mapping'
#                 or run.tags[s] == 'contrast'
#                 or run.tags[s] == 'disengaged'
#                 or run.tags[s] == 'retinotopy']

#     # check for undesirable run_type (where animal is not training)
#     type_flag = run.run_type == 'naive'

#     # update metadata for all approved runs (tag_flag == empty)
#     # add current run_type to tags and make update run_type
#     if not tag_flag and not type_flag:
#         run_type = 'training'
#     else:
#         run_type = 'other'
    
    print(run.run_type)
#     print(run_type)
    print(run.tags)

other
(u'kelly', u'naive', u'retinotopy')
other
(u'kelly', u'naive', u'orientation_mapping')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'kelly', u'naive', u'orientation_mapping')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'kelly', u'naive', u'orientation_mapping')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'kelly', u'naive', u'orientation_mapping')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'naive')
other
(u'hungry', u'kelly', u'learning_start', u'naive')
other
(u'hungry', u'kelly', u'learning_start', u'naive')
training
(u'hungry', u'kelly', u

In [7]:
i = 7
# tags = ['hungry', 'kelly', 'learning_start']
flow.metadata.add_run(runs[i].mouse, runs[i].date, runs[i].run, runs[i].run_type,
                      tags=['naive'], overwrite=False, update=True)

In [11]:
mouse = 'OA27'
update = True

# get all runs for an existing mouse
runs = flow.metadata.RunSorter.frommeta(mice=[mouse])

for run in runs:

    # check for undesirable tags (runs where animal is not training)
    tags = [str(run.tags[s]) for s in range(len(run.tags)) if len(run.tags[s]) > 1]
#             or run.tags[s] == 'naive'
#             or run.tags[s] == 'orientation_mapping'
#             or run.tags[s] == 'contrast'
#             or run.tags[s] == 'disengaged'
#             or run.tags[s] == 'retinotopy']
    print(tags)
    # update metadata for all approved runs (tag_flag == empty)
    # add current run_type to tags and make update run_type
#     print(run.run_type)
#     if not tag_flag:
#         run_type = 'training'
    flow.metadata.add_run(run.mouse, run.date, run.run, run.run_type,
                          tags=tags, overwrite=True, update=False)
#         print(run_type)
# #         print([run.run_type])
#     else:
#         run_type = 'other'
# #         flow.metadata.add_run(run.mouse, run.date, run.run, run_type,
# #                               tags=[run.run_type], overwrite=False, update=update)
#         print(run_type)
#         print([run.run_type])

['kelly', 'naive', 'orientation_mapping']
['hungry', 'kelly', 'naive']
['kelly', 'naive', 'orientation_mapping']
['hungry', 'kelly', 'naive']
['hungry', 'kelly', 'naive']
['contrast', 'kelly', 'naive']
['kelly', 'learning_start', 'orientation_mapping']
['hungry', 'kelly', 'learning_start']
['hungry', 'kelly', 'learning_start']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']
['hungry', 'kelly', 'learning']

In [39]:
print(run_type)
print(hunger)
print(tags)

naive
['hungry']
hungry


In [113]:
# tag_options = ['hungry', 'sated', 'disengaged', 'contrast', 'orientation_mapping', 'retinotopy']
# TODO add a seconds dataframe which is the trial variavles  

mouse = 'OA27'
runs = flow.metadata.RunSorter.frommeta(mice=[mouse])

# triggering parameters
start_time = -1
end_time = 6
trace_type = 'dff'

trial_list = []
for run in runs:
    
    # get your t2p object
    t2p = run.trace2p()
    
    # trigger all trials around stimulus onsets to get trial number
    run_traces = t2p.cstraces('', start_s=start_time, end_s=end_time, trace_type=trace_type,
                    cutoff_before_lick_ms=-1, errortrials=-1, baseline=None,
                    baseline_to_stimulus=True)
    trial_idx = range(np.shape(run_traces)[2])

    # get your runtype and relevant tags 
    run_type = run.run_type
    run_type = [run_type]*len(trial_idx)
    
    # get hunger state for all trials, consider hungry if not sated
    run_tags = run.tags
    hunger = [ 'hungry' for s in range(len(run_tags)) if run_tags[s] == 'hungry']
    if hunger == []:
        hunger = [ 'hungry' for s in range(len(run_tags)) if run_tags[s] != 'sated']
    hunger = np.unique(hunger)[0]
    hunger = [hunger]*len(trial_idx)
    
    # get relevant trial-distinguising tags excluding kelly, hunger state, and run_type
    tags = [ run_tags[s] for s in range(len(run_tags)) if run_tags[s] != hunger[0]
            and run_tags[s] != 'kelly'
            and run_tags[s] != run_type[0]]
    if tags == []: # define as "standard" if the run is not another option
        tags = ['standard']
    tags = tags[0]
    print(tags)
    tags = [tags]*len(trial_idx)
    
    # get trialerror ensureing you don't include runthrough at end of trials
    trialerror = np.array(t2p.d['trialerror'][trial_idx])
    
    # get cs and orientation infor for each trial 
    oris = []
    css = []
    for trial in t2p.d['condition'][trial_idx]:
        # get cs and ori
        codename = t2p.d['codes'].keys()[t2p.d['codes'].values().index(trial)]
        oriname = t2p.d['orientations'][codename]
        css.append(codename)
        oris.append(oriname)
    
    # get mean running speed for time stim is on screen
    all_onsets = t2p.csonsets()
    all_offsets = t2p.d['offsets'][0:len(all_onsets)]
    if t2p.d['running'].size > 0:
        speed_vec = t2p.speed()
        speed_vec = speed_vec.astype('float')
        speed = []
        for s in trial_idx:
            try:
                speed.append(np.nanmean(speed_vec[all_onsets[s]:all_offsets[s]]))
            except:
                speed.append(np.nan)
        speed = np.array(speed)
    else:
        speed = np.full(len(trial_idx), np.nan)

    # get offset relative to triggered data
#     offsets = all_offsets - all_onsets + (np.abs(start_time)*np.round(t2p.d['framerate']))
#     offsets = offsets.flatten()
    
    # get ensure relative to triggered data 
    ensure = t2p.ensure()
    ensure = ensure.astype('float')
    ensure[ensure == 0] = np.nan
    ensure = ensure - all_onsets + (np.abs(start_time)*np.round(t2p.d['framerate']))
    
    # get quinine relative to triggered data 
    quinine = t2p.quinine()
    quinine = quinine.astype('float')
    quinine[quinine == 0] = np.nan
    quinine = quinine - all_onsets + (np.abs(start_time)*np.round(t2p.d['framerate']))

    # get firstlick for trial
    firstlick = t2p.firstlick('')[trial_idx]
    firstlick = firstlick + (np.abs(start_time)*np.round(t2p.d['framerate']))
    
#     print(np.shape([run.mouse]*len(trial_idx)))
#     print(np.shape(trial_idx))
#     print(np.shape(oris))
#     print(np.shape(css))
#     print(np.shape(trialerror))
#     print(np.shape(hunger))
#     print(np.shape(run_type))
#     print(np.shape(tags))
#     print(np.shape(firstlick))
#     print(np.shape(ensure))
#     print(np.shape(quinine))
#     print(np.shape(speed))
    # create your index out of relevant variables 
    index = pd.MultiIndex.from_arrays([
                [run.mouse]*len(trial_idx), 
                [run.date]*len(trial_idx),
                [run.run]*len(trial_idx),
                trial_idx
                ],
                names=['mouse', 'date', 'run', 'trial_idx'])

    data = {'orientation':  oris, 'condition': css,
            'trialerror': trialerror, 'hunger': hunger,
            'run_type': run_type, 'tag': tags, 
            'firstlick': firstlick, 'ensure': ensure,
            'quinine': quinine, 'speed': speed}

    # append all trials across all runs together into a list
    trial_list.append(pd.DataFrame(data,index=index))
    
    # clear your t2p to save RAM
    run._t2p = None
    print('Run: ' + str(run) + ': ' + str(len(trial_list)))

# concatenate all runs together in final dataframe 
trial_df = pd.concat(trial_list, axis=0)

# save
save_path = os.path.join(flow.paths.outd, str(runs[0].mouse) + '_df_' + trace_type + '_trialmeta.pkl')
trial_df.to_pickle(save_path)

orientation_mapping
Run: OA27_170116_2: 1
standard
Run: OA27_170116_3: 2
orientation_mapping
Run: OA27_170118_2: 3
standard
Run: OA27_170118_3: 4
standard
Run: OA27_170119_1: 5
contrast
Run: OA27_170119_2: 6
learning_start
Run: OA27_170125_2: 7
learning_start
Run: OA27_170125_3: 8
learning_start
Run: OA27_170125_4: 9
standard
Run: OA27_170127_2: 10
standard
Run: OA27_170127_3: 11
standard
Run: OA27_170128_2: 12
standard
Run: OA27_170128_3: 13
standard
Run: OA27_170128_4: 14
standard
Run: OA27_170129_2: 15
standard
Run: OA27_170129_3: 16
standard
Run: OA27_170129_4: 17
standard
Run: OA27_170129_5: 18
standard
Run: OA27_170131_4: 19
standard
Run: OA27_170131_5: 20
standard
Run: OA27_170131_6: 21
standard
Run: OA27_170201_3: 22
standard
Run: OA27_170201_4: 23
standard
Run: OA27_170201_5: 24
standard
Run: OA27_170201_6: 25
standard
Run: OA27_170201_7: 26
standard
Run: OA27_170202_2: 27
standard
Run: OA27_170202_3: 28
standard
Run: OA27_170202_4: 29
standard
Run: OA27_170202_5: 30
standard


In [112]:
trial_df = pd.concat(trial_list, axis=0)
trial_df[10000:10010]

condition  ensure  firstlick  hunger  orientation  \
mouse date   run trial_idx                                                      
OA27  170216 2   128        pavlovian   128.0       35.0  hungry          270   
                 129            minus     NaN       57.0  hungry          135   
                 130          neutral     NaN        NaN  hungry            0   
                 131             plus     NaN        NaN  hungry          270   
                 132             plus     NaN        NaN  hungry          270   
                 133             plus     NaN        NaN  hungry          270   
                 134            minus     NaN       54.0  hungry          135   
                 135          neutral     NaN        NaN  hungry            0   
                 136            minus     NaN        NaN  hungry          135   
                 137            minus     NaN      177.0  hungry          135   

                            quinine  run_type     speed       tag  trialerror  
mouse date   run trial_idx                                                     
OA27  170216 2   128            NaN  learning  0.000000  standard           8  
                 129          132.0  learning  0.262882  standard           5  
                 130            NaN  learning  0.000000  standard           2  
                 131            NaN  learning  0.000000  standard           1  
                 132            NaN  learning  0.000000  standard           1  
                 133            NaN  learning  0.000000  standard           1  
                 134          132.0  learning  0.361323  standard           5  
                 135            NaN  learning  0.000000  standard           2  
                 136            NaN  learning  0.000000  standard           4  
                 137          178.0  learning  0.000000  standard           5